## Orchestratge LLMs

Give the same exercice to multiple LLMs and ask one LLM to classify the answers by most pertinent

In [ ]:
# Start with imports - ask ChatGPT to explain any package that you don't know

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

In [ ]:
# Always remember to do this!
load_dotenv(override=True)

python-dotenv could not parse statement starting at line 1


True

In [ ]:
# Print the key prefixes to help with any debugging

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set (and this is optional)
Google API Key not set (and this is optional)
DeepSeek API Key exists and begins sk-
Groq API Key not set (and this is optional)


In [ ]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [ ]:
messages

[{'role': 'user',
  'content': 'Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. Answer only with the question, no explanation.'}]

In [ ]:
openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
question = response.choices[0].message.content
print(question)


If you could design a society that prioritizes both individual freedom and collective responsibility, what fundamental principles and structures would you implement to ensure a balance between these two values, and how would you resolve potential conflicts that arise between them?


In [ ]:
competitors = []
answers = []
messages = [{"role": "user", "content": question}]

In [ ]:
# The API we know well

model_name = "gpt-4o-mini"

response = openai.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Designing a society that effectively balances individual freedom and collective responsibility is a complex endeavor that requires careful consideration of various fundamental principles and structures. Here’s a framework that could potentially achieve this balance:

### Fundamental Principles

1. **Mutual Respect and Dignity**: At the core of the society would be the belief in the inherent worth of every individual. This principle would foster an environment of respect, minimizing conflicts stemming from disrespect or dehumanization.

2. **Rights and Responsibilities**: Individual freedoms would be enshrined in a Bill of Rights, which would outline personal liberties such as freedom of speech, belief, and assembly. These rights would be accompanied by clear responsibilities to the community, emphasizing that with freedom comes the obligation to respect the freedoms and rights of others.

3. **Participatory Governance**: A democratic model of governance would empower citizens to participate actively in decision-making processes. This would include not only voting but also engaging in community discussions and forums that prioritize diverse voices.

4. **Local Autonomy and Decentralization**: While there would be overarching national laws, local communities would have the autonomy to establish rules and practices that best reflect their values and needs, facilitating a balance between individual choices and collective norms.

5. **Social Solidarity**: The society would cultivate a culture of solidarity, promoting the idea that collective welfare enhances individual well-being. This includes ensuring access to education, healthcare, and social services funded by a fair taxation system, fostering a sense of shared responsibility.

### Structures

1. **Community Councils**: Local councils would be established to handle community needs and conflicts. These councils would be composed of elected representatives and stakeholders, ensuring that decisions reflect the diverse interests of individuals.

2. **Conflict Resolution Mechanisms**: Mediation and restorative justice practices would be employed to address conflicts between individual freedoms and collective responsibilities. This approach emphasizes dialogue and understanding rather than punitive measures.

3. **Education for Civic Engagement**: The educational system would prioritize critical thinking, empathy, and civic responsibility from a young age. This would create a citizenry that understands the balance between personal freedoms and community obligations.

4. **Flexible Social Policies**: Policies would be designed to adapt to changing societal needs, allowing for individualized solutions that respect personal freedoms while supporting collective welfare, such as universal basic income coupled with community-based programs.

5. **Technology and Transparency**: Leveraging technology to create transparent systems of governance helps build trust within the society. Open data platforms would allow citizens to hold leaders accountable and engage more fully in the democratic process.

### Resolving Conflicts

1. **Dialogue and Deliberation**: Establish regular community forums where individuals can voice concerns and engage in, facilitated discussions that seek common ground when conflicts arise between personal freedoms and public responsibilities.

2. **Resilience Frameworks**: Create frameworks that allow for quick adaptations to policies as societal values evolve. This involves monitoring societal health indicators to anticipate tensions and adjust accordingly.

3. **Priority of Fundamental Rights**: In instances where individual rights conflict with community needs, a framework for evaluating these conflicts would prioritize fundamental human rights as laid out in the society’s constitution, ensuring these rights are minimally abridged.

4. **Stakeholder Inclusion**: In any decision-making process that could lead to conflict, all stakeholders—especially marginalized voices—should be included to ensure a fair representation of interests and perspectives.

5. **Educational Campaigns**: Promote awareness campaigns that educate citizens about both sides of conflicts, promoting understanding and cooperation between individuals and the collective.

By implementing these principles and structures, the society could foster an environment where individual freedoms are not only respected but are understood as interconnected with the collective well-being, creating a tapestry of shared responsibility and personal liberty.

In [ ]:
# Anthropic has a slightly different API, and Max Tokens is required

model_name = "claude-3-7-sonnet-latest"

claude = Anthropic()
response = claude.messages.create(model=model_name, messages=messages, max_tokens=1000)
answer = response.content[0].text

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"

response = gemini.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
model_name = "deepseek-chat"

response = deepseek.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Of course. This is one of the most profound and enduring challenges of political philosophy. Designing a society that balances individual freedom and collective responsibility requires a foundational framework that sees them not as opposing forces, but as mutually reinforcing concepts.

Here is a blueprint for such a society, built on fundamental principles, implemented through specific structures, and equipped with a conflict resolution mechanism.

### Fundamental Principles

These principles would be enshrined in a foundational charter or constitution, serving as the guiding ethos for all laws and institutions.

1.  **The Principle of Reciprocal Liberty:** Your freedom ends where it would materially harm or undermine the essential liberties of others. This moves beyond a simple "your right to swing your fist ends at my nose" to include harm to one's life, health, dignity, and access to fundamental opportunities.
2.  **The Principle of Universal Baseline:** Every individual is entitled to a baseline of well-being that enables effective freedom. This includes access to healthcare, education, nutrition, housing, and security. Without this baseline, freedom is merely theoretical for the disadvantaged.
3.  **The Principle of Subsidiarity and Sphere Sovereignty:** Decisions should be made at the most local level possible. Families, communities, local governments, and professional associations should have autonomy over their own spheres. The central government's role is to protect rights, manage national-scale issues, and ensure the local units do not become oppressive.
4.  **The Principle of Democratic Empowerment:** All individuals must have a meaningful voice in the decisions that affect their lives, not just through periodic elections but through continuous deliberative processes.
5.  **The Principle of Stewardship:** Both individuals and the collective have a responsibility to be stewards of the environment, public resources, and cultural heritage for future generations.

---

### Implementing Structures

These principles would be brought to life through a multi-layered system of governance and social organization.

**1. A Robust, Multi-tiered Democratic System:**
*   **Liquid Democracy:** A hybrid system where citizens can vote on issues directly *or* delegate their vote to a trusted representative on specific topics (e.g., a scientist for environmental policy, a teacher for education). This combines direct democracy's engagement with representative democracy's efficiency.
*   **Citizens' Assemblies:** Randomly selected, demographically representative groups of citizens would be convened to study complex issues (e.g., urban planning, ethical tech) and make binding recommendations or decisions. This counters short-term populism and special interest influence.
*   **Strong Local Governance:** Cities and regions would have significant fiscal and legislative autonomy, allowing for policy experimentation (a "laboratory of democracy" model) that aligns with local values while adhering to the fundamental principles.

**2. The Social Covenant Economy:**
*   **Market-Based, but Not Market-Driven:** A vibrant private sector would be encouraged for innovation and efficiency, but within a strict framework.
*   **Co-determination:** Workers in companies above a certain size would elect a significant portion (e.g., 40-50%) of the corporate board of directors, ensuring that profit motives are balanced with worker welfare and long-term community interests.
*   **Wealth & Resource Caps/Ceilings:** To prevent the corrosive accumulation of power that undermines both democracy and equality of opportunity, there would be a high but firm ceiling on personal wealth and a "carbon budget" for individuals and corporations.
*   **Universal Basic Endowment (UBE):** Instead of just a basic income, every citizen upon reaching adulthood would receive a substantial, one-time endowment of capital (funded by a progressive inheritance/wealth tax) to invest in education, start a business, or buy a home. This empowers individual agency from the start.

**3. The Justice and Reconciliation System:**
*   **A Multi-layered Legal Framework:** The legal system would explicitly weigh individual rights against collective responsibilities, using the fundamental principles as its guide.
*   **Restorative Justice:** For many civil and criminal conflicts, the focus would be on repairing harm and reintegrating offenders, rather than purely punitive measures, strengthening communal bonds.
*   **An Independent Constitutional Court:** A high court, insulated from political cycles, would be the ultimate arbiter of conflicts, ensuring all laws and government actions comply with the foundational principles.

**4. The Civic Infrastructure:**
*   **Mandatory National Service:** A "Service Year" would be required for all young adults, but with diverse options: military, environmental conservation, eldercare, education, or community arts. This fosters a shared sense of responsibility and connects people from different backgrounds.
*   **A Revitalized Public Sphere:** Publicly funded, independent media, community centers, and digital platforms designed for civil deliberation would be prioritized to foster a well-informed and engaged citizenry.

---

### Resolving Conflicts Between Freedom and Responsibility

Conflicts are inevitable. The system would resolve them through a clear, tiered process:

**Step 1: Local Deliberation & Mediation**
*   Most conflicts begin here. A community board or local council would use facilitated dialogue to find a compromise. For example, a conflict between a homeowner's right to modify their property (freedom) and a neighborhood's historic preservation standards (collective responsibility) would first be mediated locally.

**Step 2: Application of the "Harm Principle" Test**
*   If local mediation fails, the case moves to a civic court. The court's first test is: **Does the exercise of individual freedom create a material, demonstrable harm to the community or the liberties of others?**
    *   *Example:*
        *   **Freedom to Not Vaccinate** vs. **Collective Responsibility for Herd Immunity**. The court would rule that choosing not to vaccinate during an outbreak causes material harm to the immunocompromised, thus the collective responsibility (mandatory vaccination) overrides the individual choice in that specific context.
        *   **Freedom of Speech** vs. **Responsibility for Public Safety**. Hate speech is protected as free expression until it directly incites imminent violence (the "imminent lawless action" test), at which point the collective responsibility for safety overrides it.

**Step 3: The "Least Restrictive Means" Test**
*   If a restriction on freedom is deemed necessary, the government must prove it is using the **least restrictive means possible** to achieve the collective goal.
    *   *Example:* To address climate change (stewardship principle), the government couldn't simply ban all cars. It would have to demonstrate that a carbon budget or a robust carbon tax is a less restrictive way to achieve the same goal than an outright ban, while also investing in public transit to provide alternatives.

**Step 4: Appeal to the Constitutional Court**
*   Any party can appeal to the high court, which would make a final ruling based solely on the society's Fundamental Principles. Its role is to ensure the system does not drift toward tyranny (of the majority or a minority) and that the balance between freedom and responsibility is maintained.

### Conclusion

This society would not be a static utopia but a dynamic, complex, and sometimes messy system. Its strength would lie not in the absence of conflict, but in its robust and principled methods for resolving it. By embedding both freedom and responsibility into its very DNA—through its principles, structures, and conflict resolution mechanisms—it would create a culture where individuals see their own fulfillment as inextricably linked to the health of the community, and where the community's purpose is to empower the flourishing of every individual within it.

In [ ]:
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "llama-3.3-70b-versatile"

response = groq.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)


## For the next cell, we will use Ollama

Ollama runs a local web service that gives an OpenAI compatible endpoint,  
and runs models locally using high performance C++ code.

If you don't have Ollama, install it here by visiting https://ollama.com then pressing Download and following the instructions.

After it's installed, you should be able to visit here: http://localhost:11434 and see the message "Ollama is running"

You might need to restart Cursor (and maybe reboot). Then open a Terminal (control+\`) and run `ollama serve`

Useful Ollama commands (run these in the terminal, or with an exclamation mark in this notebook):

`ollama pull <model_name>` downloads a model locally  
`ollama ls` lists all the models you've downloaded  
`ollama rm <model_name>` deletes the specified model from your downloads

In [ ]:
!ollama pull llama3.2

In [10]:
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name = "llama3"

response = ollama.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Designing a society that balances individual freedom with collective responsibility requires careful consideration of several fundamental principles and structural elements. Here's a proposal for such a system:

**Fundamental Principles:**

1. **Human Dignity**: Every individual deserves respect, dignity, and the opportunity to reach their full potential.
2. **Equal Rights and Opportunities**: All individuals should have equal access to education, healthcare, and economic opportunities.
3. **Collective Well-being**: The collective well-being of society is essential for ensuring individual freedom and happiness.
4. **Personal Responsibility**: Individuals are responsible for their actions and decisions, while also acknowledging the impact on others.
5. **Transparency and Accountability**: Decision-making processes should be transparent, and those in power should be accountable to the people.

**Structural Elements:**

1. **Constitutional Framework**: Establish a constitutional framework that outlines individual rights and collective responsibilities, ensuring balance between personal liberty and community welfare.
2. **Representative Democracy**: Implement a representative democracy with elected officials holding accountability to the people, rather than powerful corporate interests.
3. **Decentralized Power Structures**: Empower local communities through decentralized governance models, which recognize their autonomy while promoting collective responsibility.
4. **Education and Socialization**: Provide comprehensive education and socialization programs that foster critical thinking, empathy, and community engagement.
5. **Inclusive Decision-Making Processes**: Encourage inclusive decision-making processes through participatory governance mechanisms, ensuring diverse voices are heard.

**Conflict Resolution Mechanisms:**

1. **Mediation and Arbitration**: Establish mediation and arbitration procedures to address individual and collective conflicts in a fair and impartial manner.
2. **Open-Ended Communication**: Foster open-ended communication channels between individuals, government, and community organizations to facilitate resolution of disputes.
3. **Cooperative Problem-Solving**: Encourage cooperative problem-solving approaches through community engagement and participatory governance, promoting collective responsibility for addressing societal challenges.
4. **Transparent Redress Mechanisms**: Establish transparent redress mechanisms for resolving conflicts related to individual freedom vs. collective responsibility, ensuring accountability and fairness.
5. **Evolutionary Governance**: Regularly evaluate and adapt the system through citizen-led forums and participatory governance processes, empowering individuals to shape the direction of society.

**Key Institutions:**

1. **Citizen Ombudsman Office**: Establish an independent Citizen Ombudsman Office to champion individual rights and freedoms, while promoting collective responsibility.
2. **Community Engagement Boards**: Create Community Engagement Boards that facilitate dialogue between citizens, government, and community organizations on issues related to individual freedom vs. collective responsibility.
3. **Education and Research Institutes**: Establish Education and Research Institutes that provide evidence-based insights on the intersections of individual freedom and collective responsibility.

To balance individual freedom with collective responsibility, society must be designed to:

1. **Recognize the interconnectedness** of individual rights with collective well-being.
2. **Foster a culture of empathy**, understanding that individual choices have consequences for others.
3. **Encourage active civic engagement**, recognizing that individuals are responsible for participating in the democratic process.

By implementing these principles, structures, and mechanisms, society can achieve a delicate balance between individual freedom and collective responsibility, allowing each to thrive and complement one another.

In [11]:
# So where are we?

print(competitors)
print(answers)


['gpt-4o-mini', 'deepseek-chat', 'llama3']
['Designing a society that effectively balances individual freedom and collective responsibility is a complex endeavor that requires careful consideration of various fundamental principles and structures. Here’s a framework that could potentially achieve this balance:\n\n### Fundamental Principles\n\n1. **Mutual Respect and Dignity**: At the core of the society would be the belief in the inherent worth of every individual. This principle would foster an environment of respect, minimizing conflicts stemming from disrespect or dehumanization.\n\n2. **Rights and Responsibilities**: Individual freedoms would be enshrined in a Bill of Rights, which would outline personal liberties such as freedom of speech, belief, and assembly. These rights would be accompanied by clear responsibilities to the community, emphasizing that with freedom comes the obligation to respect the freedoms and rights of others.\n\n3. **Participatory Governance**: A democratic

In [12]:
# It's nice to know how to use "zip"
for competitor, answer in zip(competitors, answers):
    print(f"Competitor: {competitor}\n\n{answer}")


Competitor: gpt-4o-mini

Designing a society that effectively balances individual freedom and collective responsibility is a complex endeavor that requires careful consideration of various fundamental principles and structures. Here’s a framework that could potentially achieve this balance:

### Fundamental Principles

1. **Mutual Respect and Dignity**: At the core of the society would be the belief in the inherent worth of every individual. This principle would foster an environment of respect, minimizing conflicts stemming from disrespect or dehumanization.

2. **Rights and Responsibilities**: Individual freedoms would be enshrined in a Bill of Rights, which would outline personal liberties such as freedom of speech, belief, and assembly. These rights would be accompanied by clear responsibilities to the community, emphasizing that with freedom comes the obligation to respect the freedoms and rights of others.

3. **Participatory Governance**: A democratic model of governance would e

In [13]:
# Let's bring this together - note the use of "enumerate"

together = ""
for index, answer in enumerate(answers):
    together += f"# Response from competitor {index+1}\n\n"
    together += answer + "\n\n"

In [14]:
print(together)

# Response from competitor 1

Designing a society that effectively balances individual freedom and collective responsibility is a complex endeavor that requires careful consideration of various fundamental principles and structures. Here’s a framework that could potentially achieve this balance:

### Fundamental Principles

1. **Mutual Respect and Dignity**: At the core of the society would be the belief in the inherent worth of every individual. This principle would foster an environment of respect, minimizing conflicts stemming from disrespect or dehumanization.

2. **Rights and Responsibilities**: Individual freedoms would be enshrined in a Bill of Rights, which would outline personal liberties such as freedom of speech, belief, and assembly. These rights would be accompanied by clear responsibilities to the community, emphasizing that with freedom comes the obligation to respect the freedoms and rights of others.

3. **Participatory Governance**: A democratic model of governance wo

In [15]:
judge = f"""You are judging a competition between {len(competitors)} competitors.
Each model has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""


In [16]:
print(judge)

You are judging a competition between 3 competitors.
Each model has been given this question:

If you could design a society that prioritizes both individual freedom and collective responsibility, what fundamental principles and structures would you implement to ensure a balance between these two values, and how would you resolve potential conflicts that arise between them?

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}

Here are the responses from each competitor:

# Response from competitor 1

Designing a society that effectively balances individual freedom and collective responsibility is a complex endeavor that requires careful consideration of various fundamental principles and structures. Here’s a framework that could potentially achieve this

In [17]:
judge_messages = [{"role": "user", "content": judge}]

In [18]:
# Judgement time!

openai = OpenAI()
response = openai.chat.completions.create(
    model="o3-mini",
    messages=judge_messages,
)
results = response.choices[0].message.content
print(results)


{"results": ["2", "1", "3"]}


In [19]:
# OK let's turn this into results!

results_dict = json.loads(results)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"Rank {index+1}: {competitor}")

Rank 1: deepseek-chat
Rank 2: gpt-4o-mini
Rank 3: llama3
